<a href="https://colab.research.google.com/github/araldi/Python_for_biomedical_data_analysis/blob/main/08_Statistics_part2_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as pl
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import scipy as sp
from scipy import stats
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('https://github.com/araldi/Python_for_biomedical_data_analysis/raw/main/Files/Homework_df_part5.csv')


In [3]:
df.head()

,Unnamed: 0,PatientID,AgeRecruit,Body_fat_percentage [%],Glucose [mmol/L],Glycated haemoglobin (HbA1c) [mmol/mol],HDL cholesterol [mmol/L],Gender,Waist circumference [cm],Hip circumference [cm],Height [cm],Diabetes diagnosed by doctor,Weight [Kg],IGF-1 [nmol/L],LDL direct [mmol/L]
0,0,1,66.0,26.2,6.309,44.7,0.913,1.0,104.0,100.0,169.0,1.0,80.8,19.684,3.241
1,1,2,46.0,27.8,NaN,34.9,NaN,0.0,66.0,93.0,159.0,0.0,53.1,NaN,NaN
2,2,3,51.0,27.2,4.888,NaN,1.514,1.0,110.0,119.0,171.0,0.0,99.5,26.463,4.384
3,3,4,57.0,19.7,4.593,37.1,1.397,1.0,87.0,99.0,165.0,0.0,72.1,17.968,2.814
4,4,5,63.0,23.0,5.333,37.0,2.060,1.0,94.0,95.0,170.0,0.0,74.2,31.057,2.525


In [4]:
# calculate BMI
# BMI formula: weight [kg] / ( height [m] * height [m])
df['BMI']= df['Weight [Kg]'] / np.power(df['Height [cm]']/100,2)

## Exercise 1

* Do females with BMI >25 kg/m^2 have higher LDL compared to the female population?

* Do females shorter than 160cm have glycated haemoglobin lower comparet do the females of the population?


Obtain the treatment sample, decide on the alpha, calculate Z-score of each sample vs population, and determined if the samples are significantly different from the population.



In [12]:
# retrieve data for glycated haemoglobin and LDL from homework part 5

females = df[(df['Gender'] == 0)]
females = females.dropna(subset= ['BMI', 'LDL direct [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]'])



* Check if females with BMI >25 kg/m^2 have higher LDL compared to the female population

In [13]:
BMI25 = females[females['BMI'] >25].sample(500)

LDL = 'LDL direct [mmol/L]'
Z_score1 = (np.mean(females[LDL]) - np.mean(BMI25[LDL]) )/(np.std(females[LDL])/sqrt(len(BMI25)))

Z_score1


-1.4374943227912553

In [14]:
import scipy
scipy.stats.norm.sf(abs(Z_score1))

0.07528879240730733

In this example, the p-value is greater than 0.05, therefore females with BMI >25 kg/m^2 DO NOT have higher LDL compared to the female population

In [15]:
from statsmodels.stats.weightstats import ztest

ztest(BMI25[LDL],value = np.mean(females[LDL]), alternative='larger')

(1.409789137163317, 0.07930097755534254)

 * Do females shorter than 160cm have glycated haemoglobin lower comparet do the females of the population?

In [16]:
height = 'Height [cm]'
glyc = 'Glycated haemoglobin (HbA1c) [mmol/mol]'

height160 = females[females[height]<160].sample(500)

Z_score2 = (np.mean(females[glyc]) - np.mean(height160[glyc]) )/(np.std(females[glyc])/sqrt(len(height160)))

scipy.stats.norm.sf(abs(Z_score2))

0.009718875836205452

Careful! Is it two tailed or one tailed? How do you adjust the p-value?


Now the easy part: verify your work with statsmodels


```
from statsmodels.stats.weightstats import ztest

ztest(sample1,value = mean, alternative='two-sided')
```



Interpretation?

Show the plots for the distributions of samples and the population (including mean).

## Exercise 2

Let's calculate a t-statistic and compute a p-value in practice for the BMI of two small samples of males and females.


Samples:


```
males = df[df['Gender'] == 1]['BMI'].sample(25)

females = df[df['Gender'] == 0]['BMI'].sample(25)

```

#### Descriptive procedure

The observed difference between samples (effect size) is a random variable, and under the null hypothesis it is zero.


```
tstat = observed difference between sample means / standard error of the difference between the means

tstat = (mean(X1) - mean(X2)) / sed

```

We need the standard error of the difference (observed effect size) among samples:

```
sed = sqrt(se1^2 + se2^2)

se = std / sqrt(n)
```



Once we obtain the tstat, obtain from the t-distribution the frequency of the obtained t-statistics, hence, the p-value

```
from scipy import stats
stats.t.sf(np.abs(tstat), n-1)*2
```
Careful! It is multiplied by two because is a two-tailed for independent samples

Lastly, validate with a pre-made function
`stats.ttest_ind(males, females)`

Show the plots for the distributions of samples (including mean).

In [18]:
males = df[df['Gender'] == 1]['BMI'].sample(25)

females = df[df['Gender'] == 0]['BMI'].sample(25)

diff = np.mean(males) - np.mean(females)
diff

-1.122730283801971

In [19]:
# the variance of the difference of two random variables is the sum of its variances

se_m = np.std(males)/sqrt(len(males))
se_f = np.std(females)/sqrt(len(females))

diff_se = sqrt(np.power(se_m, 2) + np.power(se_f, 2))

In [20]:
tstat = diff/diff_se

In [23]:
from scipy import stats
#obtain from the t-distribution the frequency of the obtained t-statistics
stats.t.sf(np.abs(tstat), len(df)-1)*2

0.39872190019558795

In [24]:
# validate with a pre-made function
stats.ttest_ind(males, females)

Ttest_indResult(statistic=-0.8268575130689035, pvalue=0.4124094622326191)

## Exercise 3
Calculate the chi-square statistics according to the formula above and then retrieve the critical value from the chi-squared distribution for the probability and number of degrees of freedom.

```
degrees_freedom = (len(rows)-1)*(len(columns)-1)
stats.distributions.chi2.sf(chi_square, degrees_freedom)
```

Lastly, validate your results with the pre-made function:

```
scipy.stats.chi2_contingency(contingency_table, 1)
```




Find an appropriate way to plot the data.

In [27]:
conditions = [df['Height [cm]']<180, df['Height [cm]']>=180]
choices = ['short', 'tall']
df['height'] = np.select(conditions, choices, default= None)

In [28]:
conditions = [df['Diabetes diagnosed by doctor']==0, df['Diabetes diagnosed by doctor']==1]
choices = ['no', 'yes']
df['diabetes'] = np.select(conditions, choices, default= None)

In [31]:
df.dropna(subset= ['height','diabetes'], inplace =True)

In [32]:
# create contingency table
data_crosstab = pd.crosstab(df['height'],
                            df['diabetes'],
                           margins=True, margins_name="Total")

In [33]:
# significance level
alpha = 0.05

# Calculation of Chisquare test statistics
chi_square = 0
rows = df['height'].unique()
columns = df['diabetes'].unique()
for i in columns:
    for j in rows:
        O = data_crosstab[i][j]
        E = data_crosstab[i]['Total'] * data_crosstab['Total'][j] / data_crosstab['Total']['Total']
        chi_square += (O-E)**2/E

# p-value obtained from the chi-square distribution
degrees_freedom = (len(rows)-1)*(len(columns)-1)

stats.distributions.chi2.sf(chi_square, degrees_freedom)

0.5973695480845165

In [34]:
import scipy
stat, p, dof, expected = scipy.stats.chi2_contingency(data_crosstab.iloc[0:2,0:2], 1)
p

0.6079907200441669